<a href="https://colab.research.google.com/github/Xavier-ML/Computational-Physics-with-Python/blob/main/07d_Mutidimensional_3_body_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multidimensional differential equations

- Börge Göbel 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from scipy import integrate

# 3-body problem

In [ ]:
# Constants in SI units

# Gravitational constant
G = 6.67430*10**(-11) # m^3 / ( kg * s^2 )

# Masses
msun = 1.9884 * 10**30 # kg
mearth = 5.9723 * 10**24 # kg
mmoon = 7.349 * 10**22 # kg

# Distances (average)
rSunEarth = 1.4960 * 10**11 # m
rEarthMoon = 3.850 * 10**8 # m

# Velocities (average)
vEarth = 29780 # m/s (Trajectory around sun)
vMoon = 1022 # m/s (Trajectory around earth)

## A) Sun, Earth & Moon

Variables for each of the 3 bodies: \\(x, y, z, v_x, v_y, v_z\\)

Force: Newton's law of gravitation 

\begin{equation}
( \vec{F} =G\frac{m_1m_2}{|\vec{r}_0-\vec{r}|^3}(\vec{r}_0-\vec{r}) = -G\frac{m_1m_2}{r^2}\vec{e}_r\end{equation} for \begin{equation} \vec{r}_0=0\end{equation}

For the three bodies

\begin{equation}
( m_1\ddot{\vec{r}_1}=
G\frac{m_1m_2}{|\vec{r}_2-\vec{r}_1|^3}(\vec{r}_2-\vec{r}_1) + G\frac{m_1m_3}{|\vec{r}_3-\vec{r}_1|^3}(\vec{r}_3-\vec{r}_1)\\
m_2\ddot{\vec{r}_2}=
G\frac{m_2m_1}{|\vec{r}_1-\vec{r}_2|^3}(\vec{r}_1-\vec{r}_2) + G\frac{m_2m_3}{|\vec{r}_3-\vec{r}_2|^3}(\vec{r}_3-\vec{r}_2)\\
m_3\ddot{\vec{r}_3}=
G\frac{m_3m_1}{|\vec{r}_1-\vec{r}_3|^3}(\vec{r}_1-\vec{r}_3) + G\frac{m_3m_2}{|\vec{r}_2-\vec{r}_3|^3}(\vec{r}_2-\vec{r}_3)
\end{equation}

We have the solve the following differential equations

\begin{equation} \ddot{\vec{r}_1}=
-G\left(\frac{m_2}{|\vec{r}_2-\vec{r}_1|^3}+\frac{m_3}{|\vec{r}_3-\vec{r}_1|^3}\right)\vec{r}_1 + G\frac{m_2}{|\vec{r}_2-\vec{r}_1|^3}\vec{r}_2 + G\frac{m_3}{|\vec{r}_3-\vec{r}_1|^3}\vec{r}_3\\
\ddot{\vec{r}_2}=
G\frac{m_1}{|\vec{r}_1-\vec{r}_2|^3}\vec{r}_1 -G\left(\frac{m_1}{|\vec{r}_1-\vec{r}_2|^3}+\frac{m_3}{|\vec{r}_3-\vec{r}_2|^3}\right)\vec{r}_2 + G\frac{m_3}{|\vec{r}_3-\vec{r}_2|^3}\vec{r}_3\\
\ddot{\vec{r}_3}=
G\frac{m_1}{|\vec{r}_1-\vec{r}_3|^3}\vec{r}_1 + G\frac{m_2}{|\vec{r}_2-\vec{r}_3|^3}\vec{r}_2 -G\left(\frac{m_1}{|\vec{r}_1-\vec{r}_3|^3}+\frac{m_2}{|\vec{r}_2-\vec{r}_3|^3}\right)\vec{r}_3
\end{equation}

### Sun

### Earth

### Moon

### Moon orbit around earth

### Exaggerate moon orbit radius

## B) Add a fourth body: Spaceship

Since the mass of the spaceship is so small compared to the other masses, we will disregard the effect on sun, earth and moon. 
The 3 differential equation remain. 

The following equation describes the spaceship

\begin{equation}
m_4\ddot{\vec{r}_4}=
G\frac{m_4m_1}{|\vec{r}_1-\vec{r}_4|^3}(\vec{r}_1-\vec{r}_4) + 
G\frac{m_4m_2}{|\vec{r}_2-\vec{r}_4|^3}(\vec{r}_2-\vec{r}_4) + 
G\frac{m_4m_3}{|\vec{r}_3-\vec{r}_4|^3}(\vec{r}_3-\vec{r}_4)
\end{equation}

We add to our systems of differential equations:

\begin{equation}
\ddot{\vec{r}_4}=
G\frac{m_1}{|\vec{r}_1-\vec{r}_4|^3}\vec{r}_1 + G\frac{m_2}{|\vec{r}_2-\vec{r}_4|^3}\vec{r}_2+ G\frac{m_3}{|\vec{r}_3-\vec{r}_4|^3}\vec{r}_3 -G\left(\frac{m_1}{|\vec{r}_1-\vec{r}_4|^3}+\frac{m_2}{|\vec{r}_2-\vec{r}_4|^3}+\frac{m_3}{|\vec{r}_3-\vec{r}_4|^3}\right)\vec{r}_4
\end{equation}

## Analyze trajectory with respect to earth and moon

### a) Elliptical orbit

### b) Direct earth escape

### c) Earth espace via moon encounter

### d) Moon orbit

We have already achieved a trajectory that aproached the moon. In order to reach a stable orbit, we must decrease the relative velocity of the spaceship once it is close to the moon. We will temporarily apply a force that breaks the spaceship relative to the moon.

The fourth differential equation changes to

\begin{equation}
\ddot{\vec{r}_4}=
G\frac{m_1}{|\vec{r}_1-\vec{r}_4|^3}\vec{r}_1 + G\frac{m_2}{|\vec{r}_2-\vec{r}_4|^3}\vec{r}_2+ G\frac{m_3}{|\vec{r}_3-\vec{r}_4|^3}\vec{r}_3 -G\left(\frac{m_1}{|\vec{r}_1-\vec{r}_4|^3}+\frac{m_2}{|\vec{r}_2-\vec{r}_4|^3}+\frac{m_3}{|\vec{r}_3-\vec{r}_4|^3}\right)\vec{r}_4 - \frac{F(t)}{m_4}\vec{e_{v_\mathrm{rel}}}
\end{equation}

with 

\begin{equation}
\vec{e_{v_\mathrm{rel}}}=\frac{\dot{\vec{r}}_4-\dot{\vec{r}}_3}{\left|\dot{\vec{r}}_4-\dot{\vec{r}}_3\right|}
\end{equation}